In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt  


In [31]:
discount_rate = 0.0541
monthly_solar_radiation = [5.9*31, 6.4*28, 7*31, 7*30, 6.8*31, 6.9*30, 7.1*31, 7.2*31, 6.7*30, 6.1*31, 5.6*30, 5.3*31]
degradation_rate = 0.00469
electricity_tariff = 0.5444
feed_in_tariff = electricity_tariff*0.50


In [37]:
eg = list(i * 10 for i in monthly_solar_radiation)
eg.reverse()
print((eg))

[1642.9999999999998, 1680.0, 1891.0, 2010.0, 2232.0, 2201.0, 2070.0, 2108.0, 2100, 2170, 1792.0000000000002, 1829.0]


In [ ]:
def graph(formula, x_range):  
    x = np.array(x_range)  
    y = formula(x)  
    plt.plot(x, y)  
    plt.show() 

In [44]:
def NPV_formula(kw, ic, d, msr, dr, et, fit, eu):
    NPV = -ic
    for i in range(25):
        yearly_value = 0
        monthly_electricity_generated = list(j * 10 * (1-dr*i) for j in msr) 
        for k in monthly_electricity_generated:
            monthly_value = -15*kw
            if k > eu:
                monthly_value += eu * et
                monthly_value += (k-eu)*fit
            else:
                monthly_value += k*et
            yearly_value += monthly_value
        NPV+=(yearly_value/(1+d)**(i+1))
    return NPV        

In [39]:
def SPP_formula(kw, ic, d, msr, dr, et, fit, eu):
    NPV = -ic
    for i in range(25):
        yearly_value = 0
        monthly_electricity_generated = list(j * 10 * (1-dr*i) for j in msr) 
        for k in monthly_electricity_generated:
            monthly_value = -15*kw
            if k > eu:
                monthly_value += eu * et
                monthly_value += (k-eu)*fit
            else:
                monthly_value += k*et
            yearly_value += monthly_value
        NPV+=(yearly_value/(1+d)**(i+1))
        if NPV > 0:
            SPP = i+1
            break
    monthly_electricity_generated.reverse()
    month_counter = 1
    for k in monthly_electricity_generated:
        month_counter -= 0.0833
        monthly_value = -15*kw
        if k > eu:
            monthly_value += eu * et
            monthly_value += (k-eu)*fit
        else:
            monthly_value += k*et
        NPV -= monthly_value/(1+d)**(i+1)
        if NPV < 0:
            SPP +=month_counter
            break
    return SPP

In [50]:
def IRR_formula(kw, ic, msr, dr, et, fit, eu):
    lower_bound = 0.0
    upper_bound = 1.0 
    tolerance = 1e-4
    max_iterations = 1000
    iteration = 0

    while iteration < max_iterations:
        mid_rate = (lower_bound + upper_bound) / 2
        npv = NPV_formula(kw, ic, mid_rate, msr, dr, et, fit, eu)

        if abs(npv) < tolerance: 
            return mid_rate
        elif npv > 0:
            lower_bound = mid_rate  
        else:
            upper_bound = mid_rate  
        
        iteration += 1

    return None

In [54]:
def LCOE_formula(kw, ic, d, msr, dr, et, fit, eu):
    NPV_costs = ic
    NPV_energy = 0
    for i in range(25):
        NPV_costs += 12*15*kw/(1+d)**(i+1)
        NPV_energy += (sum(msr) * 10 * (1 - dr*i)) / (1+d)**(i+1)
    return NPV_costs/NPV_energy
    

In [61]:
print(f"{round(IRR_formula(10, 19500, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, 1000),4)*100}%")

40.25%


In [64]:
print(f"{round(SPP_formula(10, 19500, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, 1000), 2)} years")


3.67 years


In [66]:
print(f"{round(LCOE_formula(10, 19500, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, 1000), 2)} AWG/kWh")

0.14 AWG/kWh


In [68]:
print(f"{round(NPV_formula(10, 19500, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, 1000), 2)} AWG")

83913.4 AWG
